In [ ]:
import os
import shutil

import papermill as pm

In [ ]:
username = "Fro116"
source = "MAL"
training_task = None
DAEMON_MODE = False

In [ ]:
if source != "Training":
    training_task = None

In [ ]:
data_path = os.path.abspath("../../data/recommendations")
if not os.path.exists(data_path):
    os.mkdir(data_path)
rec_dir = os.path.join(data_path, username)
papermill_path = os.path.join(rec_dir, "papermill")

In [ ]:
julia_args = "-e 'using DaemonMode; runargs({port})'"
if not DAEMON_MODE:
    print(
        "Julia daemon not detected, disabling client mode. Run the "
        "following commands and set DAEMON_MODE=true to enable julia "
        "daemon mode:"
    )
    for i in range(6):
        print(f"julia -e 'using DaemonMode; serve(300{i}, true)' &")
    julia_args = ""

In [ ]:
def run_julia_notebook(name, port, *args):
    os.system(f"julia {julia_args.format(port=port)} {name} {' '.join(args)}")

## RecIngress

In [ ]:
def run_ingress_notebook(name, **kwargs):
    outdir = os.path.join(papermill_path, "rec_ingress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    pm.execute_notebook(
        f"../RecIngress/{name}.ipynb",
        f"{outdir}/{name}.ipynb",
        parameters=dict(username=username, source=source, **kwargs),
    )

In [ ]:
def run_ingress_julia(name, *args):
    cwd = os.getcwd()
    os.chdir("../RecIngress")
    for folder in name.split("/")[:-1]:
        os.chdir(folder)
    try:
        run_julia_notebook(f"{name.split('/')[-1]}.jl", 3000, username, *args)
    finally:
        os.chdir(cwd)

In [ ]:
if os.path.exists(rec_dir):
    shutil.rmtree(rec_dir)

In [ ]:
# TODO consolidate papermill startup costs

In [ ]:
run_ingress_notebook("FetchAnimeLists", training_task=training_task)

In [ ]:
run_ingress_notebook("ImportAnimeLists")

In [ ]:
run_ingress_notebook("ProcessAnimeLists")

In [ ]:
run_ingress_notebook("RelabelIds")

In [ ]:
run_ingress_notebook("GenerateSplits")

In [ ]:
run_ingress_julia("CompressSplits")

## InferenceAlphas

In [ ]:
def run_inference_julia(name, port, *args):
    cwd = os.getcwd()
    os.chdir("../InferenceAlphas")
    for folder in name.split("/")[:-1]:
        os.chdir(folder)
    try:
        run_julia_notebook(f"{name.split('/')[-1]}.jl", port, username, *args)
    finally:
        os.chdir(cwd)

In [ ]:
run_inference_julia("Explicit/Explicit", 3001)

In [ ]:
run_inference_julia("Nondirectional/Nondirectional", 3002)

In [ ]:
run_inference_julia("Transformer/Transformer", 3003)

In [ ]:
run_inference_julia("Neural/Neural", 3004)

In [ ]:
run_inference_julia("Ensemble/Ensemble", 3005)

## RecEgress

In [ ]:
def run_egress_notebook(name, **kwargs):
    outdir = rec_dir
    pm.execute_notebook(
        f"{name}.ipynb",
        f"{outdir}/{name}.ipynb",
        parameters=dict(username=username, source=source, **kwargs),
    )

In [ ]:
if source == "Training":
    run_egress_notebook("TrainingServingSkew", task=training_task)
else:
    run_egress_notebook("Recommendations")

In [ ]:
print("Success!")